# Amazon urbanization project analysis protocol

Contributors: Laura-Isobel McCall, Chris Callewaert, Qiyun Zhu, Se Jin Song, James T. Morton

## Background

 - Qiita study ID: [10333](https://qiita.ucsd.edu/study/description/10333) (title: *Dominguez Sloan SAWesternization gradient*).
 - Barnacle project directory: `sloan_10333`
 - EBI data deposition: [ERP107551](https://www.ebi.ac.uk/ena/data/view/ERP107551).

## DNA data analysis

### Information

Qiita prep IDs:
 - 16S: 1227, 1228, 1229, 1234
 - 18S: 1243
 - ITS: 1235

For 16S, dowload auto-deblurred BIOM tables from Qiita
 - Against Greengenes release 13_8, 88% OTU
 - dflt_30888, dflt_30777, dflt_30890, dflt_30585

For 18S and ITS data, run deblur locally on proper databases.
 - Using deblur version 1.0.2
 - 18S against Silva release 123, 80% OTU
 - ITS against UNITE release 7.1, 97% OTU
 
Note: These database releases were chosen because they were already deployed in Barnacle. To download them fresh, the links are [18S](https://www.arb-silva.de/fileadmin/silva_databases/qiime/Silva_123_release.zip) and [ITS](https://unite.ut.ee/sh_files/sh_qiime_release_s_22.08.2016.zip). The guideline in the QIIME [website](http://qiime.org/home_static/dataFiles.html) was followed in selecting databases.
 
Note: The deblur parameters were set following the default setting in Qiita.

### Pre-processing

In [ ]:
%%bash

# 16S: 4 preps (already automated in Qiita)
pos_ref_fp=/databases/gg/13_8/rep_set/88_otus.fasta
pos_ref_db_fp=/databases/gg/13_8/sortmerna/88_otus

# 18S: dflt_29852
pos_ref_fp=/databases/silva_18s/silva123/silva_18s/80_otus_18S.fasta
pos_ref_db_fp=/databases/silva_18s/silva123/silva_18S/80_otus_18S

# ITS: dflt_29828
pos_ref_fp=/databases/unite/7_1/sh_refs_qiime_ver7_97_s_22.08.2016.fasta
pos_ref_db_fp=/databases/unite/7_1/unite_ITS

In [ ]:
%%bash
source activate deblurenv

deblur workflow \
  --seqs-fp seqs.fasta \
  --output-dir $outdir \
  --trim-length -1 \
  --pos-ref-fp ${pos_ref_fp} \
  --pos-ref-db-fp ${pos_ref_db_fp} \
  --min-reads 0

Drop all blank samples, and translate sample IDs to a simpler, uniform format.

In [ ]:
%%python
from biom import load_table
from biom.util import biom_open

table = load_table('16S/prep_1227/dflt_30888.biom')

ids_to_keep = set([x for x in table.ids() if not 'blank' in x.lower()])
table.filter(ids_to_keep=ids_to_keep, inplace=True)

with open('id_map.txt', 'r') as f:
    id_map = dict(x.split('\t') for x in f.read().splitlines())
table.update_ids(id_map=id_map)

with biom_open('prep_1227.biom', 'w') as f:
    table.to_hdf5(f, table.generated_by)

Merge the four 16S BIOM tables into one.

In [ ]:
%%bash
merge_otu_tables.py \
  --input_fps prep_1227,prep_1228.biom,prep_1229.biom,prep_1234.biom \
  --output_fp 16S.biom

### Taxonomic assignment

Get sequences from BIOM tables

In [ ]:
%%bash
biom convert --to-tsv -i 16S.biom -o 16S.tsv
while read line
do
  echo '>'$line >> 16S.fa
  echo $line >> 16S.fa
done < <(cat 16S.tsv | grep -v '#' | cut -f1)

Reference databases (the finest clustering scheme (99%) was used).

In [ ]:
%%bash

# 16S:
reference_seqs_fp=/databases/gg/13_8/rep_set/99_otus.fasta
id_to_taxonomy_fp=/databases/gg/13_8/taxonomy/99_otu_taxonomy.txt

# 18S:
reference_seqs_fp=rep_set/rep_set_18S_only/99/99_otus_18S.fasta
id_to_taxonomy_fp=taxonomy/18S_only/99/taxonomy_7_levels.txt

# ITS:
reference_seqs_fp=sh_refs_qiime_ver7_99_s_22.08.2016.fasta
id_to_taxonomy_fp=sh_taxonomy_qiime_ver7_99_s_22.08.2016.txt

Assign taxonomy using the [SortMeRNA](https://github.com/biocore/sortmerna) ([Kopylova, Noé and Touzet, 2012](https://academic.oup.com/bioinformatics/article/28/24/3211/246053)) method.

In [ ]:
%%bash
assign_taxonomy.py \
  --input_fasta_fp 16S.fa \
  --output_dir 16S \
  --reference_seqs_fp ${reference_seqs_fp} \
  --id_to_taxonomy_fp ${id_to_taxonomy_fp} \
  --assignment_method sortmerna

Check assignment ratio

In [ ]:
%%bash
# total sequences
cat 16S/16S_tax_assignments.txt | tail -n+2 | wc -l
# unassigned sequences
cat 16S/16S_tax_assignments.txt | tail -n+2 | grep $'\t'Unassigned$'\t' | wc -l

Unassignment ratios:
 - 16S: 7501 / 210370 = 3.56%
 - 18S: 6688 / 29778 = 22.46%
 - ITS: 5032 / 47062 = 10.69%

Append taxonomy to BIOM tables

In [ ]:
%%bash
biom add-metadata \
  --input-fp 16S.biom \
  --output-fp 16S.wtax.biom \
  --observation-metadata-fp 16S/16S_tax_assignments.txt \
  --observation-header OTUID,taxonomy \
  --sc-separated taxonomy

There were non-standard characters in the ITS assignment result (specifically, `s__Montagnula_aloës`), which caused error running biom add-metdata. We followed the protocol [here](https://groups.google.com/forum/#!topic/qiime-forum/W6NqdoWhNfI) to resolve the issue.

### Post-processing

For 16S, perform bloom-filtering, using the script and references provided in [Amir et al. (2017)](http://msystems.asm.org/content/2/2/e00199-16).

In [ ]:
%%bash
python filterbiomseqs.py -i 16S.biom -o 16S.bf.biom -f newbloom.10.fna

But there was no bloom sequences found. So this step was omitted.

Filter out sequences with <10 counts study-wide.

In [ ]:
%%bash
filter_otus_from_otu_table.py -i 16S.biom -o 16S.n10.biom -n 10
filter_otus_from_otu_table.py -i 16S.wtax.biom -o 16S.wtax.n10.biom -n 10

For 18S, perform taxonomic filterings.

In [ ]:
%%bash
# no fungi
filter_taxa_from_otu_table.py -i 18S.biom -n "D_3__Fungi" -o 18S.noFungi.biom
# animals only
filter_taxa_from_otu_table.py -i 18S.biom -p "D_3__Metazoa (Animalia)" -o 18S.animals.biom
# plants only (green algae and land plants)
filter_taxa_from_otu_table.py -i 18S.biom -p "D_2__Chloroplastida" -o 18S.plants.biom
# no animal, plants and fungi
filter_taxa_from_otu_table.py -i 18S.biom -n "D_3__Fungi,D_3__Metazoa (Animalia),D_2__Chloroplastida" -o 18S.noAPF.biom

Perforn rarefaction, to a sampling depth of 1000.

In [ ]:
%%bash
filter_samples_from_otu_table.py -i 16S.biom -o 16S.mc1000.biom -n 1000
single_rarefaction.py -i 16S.mc1000.biom -o 16S.even1000.biom -d 1000

From this point on, all subsequent analyses were based on `16S.even1000.biom`, unless otherwise stated.

Filter human vs house samples.

In [ ]:
%%bash
filter_samples_from_otu_table.py -i 16S.biom -m metadata.txt -s 'host_type:human' -o 16S.human.biom
filter_samples_from_otu_table.py -i 16S.biom -m metadata.txt -s 'host_type:house' -o 16S.house.biom

Taxonomic profiling.

In [ ]:
%%bash
sort_otu_table.py -i 16S.biom -o 16S.sorted.biom
summarize_taxa.py -i 16S.sorted.biom -o 16S

### Alpha diversity

In [ ]:
%%bash
multiple_rarefactions.py -i 16S.mc1000.biom -m 10 -x 1000 -s 99 -o 16S.multi
alpha_diversity.py -i 16S.multi -o 16S.alpha --metrics observed_otus,chao1,shannon
collate_alpha.py -i 16S.alpha -o 16S
rm -rf 16S.alpha 16S.multi

### Beta diversity

In [ ]:
%%bash
for metric in bray_curtis abund_jaccard
do
    beta_diversity.py --metrics $metric -i 16S.biom -o .
    principal_coordinates.py -i ${metric}_16S.txt -o ${metric}_16S.pcoa
    make_emperor.py -m metadata.tsv -i ${metric}_16S.pcoa -o ${metric}_16S.emp1
done

Visualization was also performed using Emperor bundled in QIIME 2.

In [ ]:
%%bash
qiime tools import --type PCoAResults --input-path ${metric}_16S.pcoa --output-path ${metric}_16S.pcoa.qza
qiime emperor plot --m-metadata-file metadata.tsv --i-pcoa ${metric}_16S.pcoa.qza --o-visualization ${metric}_16S.pcoa.emp2.qzv

Supervised classification, using the [random forest](https://en.wikipedia.org/wiki/Random_forest) method.

In [7]:
%%bash
supervised_learning.py \
  --input_data 16S.biom \
  --output_fp $category \
  --mapping_fp $category.txt \
  --category $category/16S

Compare categories, using the [adonis](http://cc.oulu.fi/~jarioksa/softhelp/vegan/html/adonis.html) method (a.k.a., PERMANOVA) as implemented in vegan 2.4-4.

In [10]:
%%bash
compare_categories.py \
  --method adonis \
  --input_dm bray_curtis_16S.txt \
  --output_dir $category/16S \
  --mapping_file $category.txt \
  --categories $category \
  --num_permutations 999

## MS data analysis

Alpha diversity: observed richness

### Beta diversity

Analyzed using the same protocol as that for DNA.

In [ ]:
%%bash
# QIIME 1
beta_diversity.py --metrics bray_curtis -i MS.biom -o .
principal_coordinates.py -i bray_curtis_MS.txt -o bray_curtis_MS.pcoa
make_emperor.py -m metadata.tsv -i bray_curtis_MS.pcoa -o bray_curtis_MS.emp1

In [ ]:
%%bash
# QIIME 2
qiime tools import --type PCoAResults --input-path bray_curtis_MS.pcoa --output-path bray_curtis_MS.pcoa.qza
qiime emperor plot --m-metadata-file metadata.tsv --i-pcoa bray_curtis_MS.pcoa.qza --o-visualization bray_curtis_MS.pcoa.emp2.qzv

## Multi-omics analysis

### Pearson correlation

Apply the Pearson correlation test on each of the three DNA feature tables, using the sum of relative abundances of each of the cleaning product categories as metadata.

In [ ]:
%%bash
observation_metadata_correlation.py -s pearson -m metadata.tsv -c MS -i 16S.biom -o 16S.MS.txt

Report the Pearson score and the Benjamini-Hochberg FDR-corrected p-value.

### PLSSVD

We applied the Partial Least Squares Singular Value Decomposition (**PLSSVD**) method ([Kapono et al., 2018](https://www.nature.com/articles/s41598-018-21541-4)) to explore the correlation between microbiome and metabolome data and with their metadata.

Source codes are under the "plssvd" directory. They were derived and modified from the [original source codes](https://github.com/knightlab-analyses/office-study/tree/master/ipynb) used in Kapono et al. (2018).

In [ ]:
%%bash
python plssvd.py \
  metadata/house.txt \
  microbes/ITS.biom.qza \
  metabolites/all.biom.qza \
  microbes/label/ITS.txt \
  metabolites/label/trim.txt \
  > ITS_all_house.log

## General statistics

### Kruskal-Wallis test

Example: Test whether the variance of 16S data among villages:

In [ ]:
%%bash
group_significance.py -s kruskal_wallis -i 16S.house.biom -m metadata.tsv -c village_socio -o output.txt

### Fishers exact test

Example: Text on `bac_match_nbrs` as the table of dataset matches per village.

In [ ]:
%%R
fisher.test(bac_match_nbrs, workspace=2000000000)

### Spearman test

### PERMANOVA

In [ ]:
%%R
library('vegan')

# read in the metadata table filtered to include human gut samples (fecal and anal), and the distance matrix.
humangut<-read.table('./metadata_human_gut.txt', h=T,row.names=1,check=F)
dm16s<-read.table('./bray_curtis_distance_matrix_16S.txt', h=T,row.names=1,check=F)

# filter the distance matrix and metadata to samples that overlap 
ix <- intersect(rownames(humangut),rownames(dm16s))
metadata<-humangut[ix,]
dist <- as.dist(as.matrix(dm16s[ix,ix])

# run permanova via the adonis function in the vegan package.
adonis16sgut<-r(dist~metadata$village_socio, data=metadata[labels(dist),])

### Wilcoxon rank sum test

Example: Representative Wilcoxon test for mass spectrometry feature m/z 318.300 RT 329 sec, comparing between Checherta and Puerto Almendra:

In [ ]:
%%R
wilcox.test(Che_PA$X318.300.329.1..ID..1269.~Che_PA$village_socio)

### Linear regression

Example: Linear regression model analysis for mass spectrometry feature m/z 287.232 RT 211 sec:

In [13]:
%%R
model<-glm(formula=full_norm_Apr25metadata_noPeruvianSwabs_housing$X287.232.211.1..ID..4006.~ full_norm_Apr25metadata_noPeruvianSwabs_housing$village_socio, family="gaussian")
summary(model)
anova(model, test="Chisq")